##### Week 18 Group Exercise
###### Angela Spencer January 26, 2022

##### 1. Look up the Adam optimization functions in PyTorch
https://pytorch.org/docs/stable/optim.html
##### How does it work? Try at least one other optimization function with the diabetes dataset shown in class. How does the model perform with the new optimizer? Did it perform better or worse than Adam? Why do you think that is?

I created a loop to iterate through each optimizer function and print the classification report for the final epoch. I found that Adadelta and RMSprop were the only to opptimizers that out-performed Adam.  The other optimizers were slightly worse or the same for accuracy metrics.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

diabetes_df = pd.read_csv("../Datasets/diabetes.csv")

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42, stratify=y)

sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#convert to tensors
X_train = torch.FloatTensor(X_train) 
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train) 
y_test = torch.LongTensor(y_test)

In [3]:
#aritficial neural network
#creating a class using the neural network module
class ANN_Model(nn.Module):
    
    #uses the parameters for nn.Module, check documentation
    def __init__(self, input_features=8, hidden1=20, hidden2=20, out_features=2):
        
        # keyword super is a computed indirect reference, 
        # iolates changes and makes sure children in the layers of multiple inheritance
        # are calling the correct parents
        super().__init__() 
        
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
        
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [4]:
torch.manual_seed(42)

#create instance of model
ann = ANN_Model()

#loss function
loss_function = nn.CrossEntropyLoss()

# list of optimizers to evaluate
optimizer_list = [torch.optim.Adadelta, torch.optim.Adagrad, torch.optim.Adam, torch.optim.AdamW,
                  torch.optim.Adamax, torch.optim.ASGD, torch.optim.NAdam, torch.optim.RAdam, 
                  torch.optim.RMSprop,torch.optim.Rprop, torch.optim.SGD]

# for loop to explore different optimizers
for x in optimizer_list:
    
    #optimizer
    optimizer = x(ann.parameters(), lr=0.1)
    
    #run model through multiple epochs/iterations
    final_loss = []
    n_epochs = 501
    for epoch in range(n_epochs):
        y_pred = ann.forward(X_train)
        loss = loss_function(y_pred, y_train)
        final_loss.append(loss)

        if epoch == 500:
            print(f'Optimizer {x}: Epoch number {epoch} with loss, {loss}')

        # impliment optimizer
        # gradient descent - zero the gradient before running backwards propagation
        optimizer.zero_grad()
        loss.backward()
        #perform optimization step for each epoch
        optimizer.step()
        
        if epoch == 500:
            #predictions 
            y_pred = []

            with torch.no_grad():
                for i, data in enumerate(X_test):
                    prediction = ann(data)
                    y_pred.append(prediction.argmax())

            print(classification_report(y_test, y_pred))

Optimizer <class 'torch.optim.adadelta.Adadelta'>: Epoch number 500 with loss, 0.4302569329738617
              precision    recall  f1-score   support

           0       0.77      0.85      0.81       150
           1       0.66      0.54      0.59        81

    accuracy                           0.74       231
   macro avg       0.72      0.69      0.70       231
weighted avg       0.73      0.74      0.73       231

Optimizer <class 'torch.optim.adagrad.Adagrad'>: Epoch number 500 with loss, 0.08438178896903992
              precision    recall  f1-score   support

           0       0.72      0.68      0.70       150
           1       0.46      0.51      0.48        81

    accuracy                           0.62       231
   macro avg       0.59      0.59      0.59       231
weighted avg       0.63      0.62      0.62       231

Optimizer <class 'torch.optim.adam.Adam'>: Epoch number 500 with loss, 0.01809709146618843
              precision    recall  f1-score   support

     

##### 2. Write a function that lists and counts the number of divisors for an input value.
    Example 1:
    Input: 5
    Output: “There are 2 divisors: 1 and 5”
    Example 2:
    Input: 40
    Output: “There are 8 divisors: 1, 2, 4, 5, 8, 10, 20, and 40”

In [5]:
import numpy as np

In [6]:
separator = ", "

def factor_fun(num):
    factors = []
    for i in np.arange(1, num+1):
        if num % i == 0:
            factors.append(i)
    else:
        pass
    return f'There are {len(factors)} factors in {num}: {(separator.join(map(str, factors[:-1])))}, and {factors[-1]}'

In [7]:
factor_fun(5)

'There are 2 factors in 5: 1, and 5'

In [8]:
factor_fun(40)

'There are 8 factors in 40: 1, 2, 4, 5, 8, 10, 20, and 40'